# <Font color = 'indianred'>**Specify Project Folder**

In [1]:
# add lines for autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys

if 'google.colab' in str(get_ipython()):
    from google.colab import drive  # Import Google Drive mounting utility
    drive.mount('/content/drive')  # Mount Google Drive

    # REPLACE WITH YOUR FOLDER

    base_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data')

    data_folder =  Path('/content')

    !pip install pytorch-lightning -U -qq
    !pip install torchmetrics -U -qq
    !pip install fastdownload -U -qq
    !pip install fastai -U -qq
    !pip install wandb -U -qq

else:
    # Set base folder path for storing files on local machine
    # REPLACE WITH YOUR FOLDER
    # FILL THIS ONLY IF YOU ARE RUNNING ON A LOCAL MACHINE
    print('Path is /Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data')

    base_folder = Path('/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data')
    data_folder = base_folder
    !pip install pytorch-lightning -U -qq
    !pip install torchmetrics -U -qq
    !pip install fastdownload -U -qq
    !pip install fastai -U -qq
    !pip install wandb -U -qq

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [7]:
# custom_function_folder = base_folder/'data/custom-functions/fall_2023'
# sys.path.append(str(custom_function_folder))
# model_folder = base_folder/'data/models/dl_fall_2023/dog_breed/oct-9'
# model_folder.mkdir(parents=True, exist_ok=True)
# project_folder = base_folder/'data/imagenette2'
# kaggle_api_folder = base_folder/'data/.kaggle'


# Change the custom_function_folder to folder in your Google drive folder
# Make sure you keep the mlp_skip_two_layer.py and shared_utils.py files
from pathlib import Path
import sys

# Determine the storage location based on the execution environment
# If running on Google Colab, use Google Drive as storage
if 'google.colab' in str(get_ipython()):
    custom_function_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Custom_files') # Your Google Drive

    sys.path.append(str(custom_function_folder))
    model_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data') # Google drive folder where you want to save model and logs
    model_folder.mkdir(parents=True, exist_ok=True)
    project_folder = base_folder/'data/imagenette2'
    kaggle_api_folder = base_folder/'data/.kaggle'

# If running locally, specify a different path
else:
    # Set base folder path for storing files on local machine
    # REPLACE WITH YOUR FOLDER
    # FILL THIS ONLY IF YOU ARE RUNNING ON A LOCAL MACHINE
    print('Path is /Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Custom_files')
    custom_function_folder = Path('/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Custom_files') # Your Google Drive

    sys.path.append(str(custom_function_folder))
    model_folder = Path('/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data') # Google drive folder where you want to save model and logs
    model_folder.mkdir(parents=True, exist_ok=True)
    project_folder = base_folder/'data/imagenette2'
    kaggle_api_folder = base_folder/'data/.kaggle'
    # project_folder = Path('/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data')


# <Font color = 'indianred'>**Import Libraries**

In [4]:
import torch
from torchvision import transforms
from pytorch_lightning.loggers import CSVLogger, WandbLogger
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.tuner import Tuner
from torch.optim.lr_scheduler import OneCycleLR

from multiclass_lightning_module_with_scheduler import MultiClassLightningModule
from data_module_dog_breed import DogBreedDataModule

from model_two_layer_bn import TwoLayerMLPBN
from shared_utils import  plot_losses_acc

import wandb

# <Font color = 'indianred'>**Setup Trainer**

## <Font color = 'indianred'>**Transformations**

In [5]:
# Define preprocessing transformations for training and testing datasets
data_transforms = {
    "train": transforms.Compose(
        [
            # Resize the shorter side of the image to (500, 375) pixels
            transforms.Resize((500, 375)),
            # Convert image to PyTorch tensor
            transforms.ToTensor(),
            # Normalize tensor values to range [-1, 1]
            transforms.Normalize((0.5,), (0.5,)),
        ]
    ),
    "test": transforms.Compose(
        [
            # Resize the shorter side of the image to 640 pixels
            transforms.Resize((500, 375)),
            # Convert image to PyTorch tensor
            transforms.ToTensor(),
            # Normalize tensor values to range [-1, 1]
            transforms.Normalize((0.5,), (0.5,)),
        ]
    ),
}

# <Font color = 'indianred'>**Setup Model, DataModule, Lightning Module**

In [8]:
# Define the model architecture and training parameters

# model hyperparameters
num_features = 3 * 500 * 375
hidden_dim1 = 500
hidden_dim2 = 500
num_classes = 120

# datamodule hyperprameters
batch_size = 1024
train_transform=data_transforms["train"]
test_transform=data_transforms["test"]
batch_size=64
seed=42
fraction_train=0.8
small_subset = True
num_samples_small = 1024

# lightning module hyperparameters
epochs=2
optimizer_cls= torch.optim.AdamW
loss_fn=torch.nn.CrossEntropyLoss()
metric_cls=torchmetrics.Accuracy
num_classes=120
learning_rate  = 0.0001
log_every_n_steps=1
log_test_metrics=True
display_metrics=True
optimizer_params={'weight_decay': 10}
scheduler_cls=OneCycleLR
scheduler_options = {'interval': 'step', 'frequency': 1}
scheduler_params = {'max_lr': 0.0005, 'total_steps':epochs * 1024//batch_size}

#logger hyperparameters
log_dir = model_folder/'logs'
project_name = 'dog_breed'
experiment_name = 'two_layer_bn'


# Set a random seed for reproducibility
torch.manual_seed(seed)

# Specify Model
model = TwoLayerMLPBN(num_features, hidden_dim1, hidden_dim2, num_classes)

# Specify DataModule
dm = DogBreedDataModule(data_dir=data_folder,
                        kaggle_api_folder=kaggle_api_folder,
                        train_transform=train_transform,
                        test_transform=test_transform,
                        batch_size=batch_size,
                        seed=seed,
                        fraction_train=fraction_train,
                        small_subset = True ,
                        num_samples_small = 1024)

# Specify LightningModule
lightning_model = MultiClassLightningModule(model=model,
                                            optimizer_cls= torch.optim.AdamW,
                                            loss_fn=torch.nn.CrossEntropyLoss(),
                                            metric_cls=torchmetrics.Accuracy,
                                            num_classes=num_classes,
                                            learning_rate  = learning_rate,
                                            log_every_n_steps=log_every_n_steps,
                                            log_test_metrics=True,
                                            display_metrics=True,
                                            optimizer_params=optimizer_params,
                                            scheduler_cls=OneCycleLR,
                                            scheduler_params=scheduler_params,
                                            scheduler_options=scheduler_options,
                                       )

## <Font color = 'indianred'>**Setup Callbacks, Logger, Trainer**

In [9]:
lr_monitor = LearningRateMonitor(logging_interval='step')

model_checkpoint_callback = ModelCheckpoint(dirpath=log_dir,
                                            save_top_k=1, mode="max", monitor="val_metric", save_last=True)
callbacks = [model_checkpoint_callback, lr_monitor ]

######### NEW #################################
wandb_logger = WandbLogger(project="Imagenette", name = 'wandbcheck1', save_dir=log_dir)
wandb_logger.experiment.config.update({'batch_size': batch_size, 'epochs': epochs})
wandb_logger.watch(model)

csv_logger = CSVLogger(save_dir=log_dir, name="two_layer")
csv_logger.log_hyperparams(params={'batch_size': batch_size, 'epochs': epochs})

##################################################


trainer = pl.Trainer(
        callbacks=callbacks,
        max_epochs=epochs,
        accelerator="auto",  # set to "auto" or "gpu" to use GPUs if available
        devices= 'auto',  # Uses all available GPUs if applicable
        deterministic=True,
        log_every_n_steps = log_every_n_steps,
        logger= [csv_logger, wandb_logger], ######## NEW HERE ##############


    )

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# <Font color = 'indianred'>**Learning Rate Finder**

In [10]:
# tuner = Tuner(trainer)
# lr_finder = tuner.lr_find(lightning_model, datamodule=dm, min_lr=1e-5, max_lr=1, num_training=30, mode='exponential')
# fig = lr_finder.plot(suggest=True)

In [11]:
# get suggestion
# new_lr = lr_finder.suggestion()
# print(new_lr)

# <Font color = 'indianred'>**Training and Evaluations**

In [12]:
trainer.fit(lightning_model, dm)

OSError: ignored

# <Font color = 'indianred'>**Plot Losses**

In [13]:
trainer.logger

In [14]:
file = f"{trainer.logger.log_dir}/metrics.csv"
plot_losses_acc(file)


FileNotFoundError: ignored

# <Font color = 'indianred'>**Evaluation**

In [ ]:
ckpt_path = trainer.checkpoint_callback.best_model_path
ckpt_path

In [ ]:
train_acc = trainer.validate(dataloaders=dm.train_dataloader(), ckpt_path=ckpt_path, verbose=False)
valid_acc = trainer.validate(dataloaders=dm.val_dataloader(), ckpt_path=ckpt_path, verbose=False)
# test_acc = trainer.test(dataloaders=dm.test_dataloader(), ckpt_path=ckpt_path, verbose=False)

In [ ]:
print(f"Train Accuracy: {train_acc[0]['val_metric']*100:0.2f}")
print(f"Validation Accuracy: {valid_acc[0]['val_metric']*100:0.2f}")
# print(f"Test Accuracy: {test_acc[0]['test_metric']*100:0.2f}")

In [ ]:
wandb.finish()